In [1]:
import os,sys,glob
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats as st
import functions as fun

In [2]:
basin = ['eais','wedd','amun','ross']
imods = ['AISM_VUB','BISI_LBL','CISM_NCA','FETI_VUB','GRIS_LSC','IMAU_VUB','ISSM_JPL','ISSM_UCI','MALI_LAN','PISM_AWI','PISM_DMI','PISM_PIK','PISM_VUW','PS3D_PSU','SICO_UHO','UA_UNN']

In [3]:
N = 20000 #20000

In [4]:
def calcSLR(TH,years,N):
    bms = np.random.uniform(7,16,N)
    ii  = np.random.randint(len(imods), size=N)
    SLRens = np.zeros((N,TH.shape[0],TH.shape[1]))
    for n in range(N):   
        for b,bb in enumerate(range(1,5)):
            with open(f'../rawdata/Larmip2019-master/RFunctions/RF_{imods[ii[n]]}_BM08_R{bb}.dat') as f:
                R = np.array([float(x) for x in f.readlines()])
            for t,tt in enumerate(years):
                if t==0: continue
                SLRens[n,b,t] = bms[n]*100*np.sum(TH[b,:t][::-1]*R[:t])
            SLRens[n,b,:] = fun.detrend(SLRens[n,b,:],np.array([]),years)
    SLR = np.sum(SLRens,axis=1)
    return SLR

In [5]:
mods = fun.models()
for scen in ['rcp45','rcp85']:
    sig = np.zeros(len(mods))
    for m,model in enumerate(mods):
        with xr.open_dataset(f'../rawdata/Adyn_temp/{model}_{scen}.nc') as ds:
            TH = ds['TH'].values
            years = ds['time'].values
        SLR = calcSLR(TH,years,N)
        sig[m] = np.std(np.nanmean(SLR[:,-20:],axis=1))
        print(scen,model,sig[m])
    print('---------------')
    print(scen,np.mean(sig))
    print('---------------')

rcp45 bcc-csm1-1 1.8383270503140041
rcp45 CanESM2 4.3441091969617815
rcp45 CCSM4 6.321348187419336
rcp45 CNRM-CM5 6.527048774677246
rcp45 CSIRO-Mk3-6-0 3.0439429009061842
rcp45 GFDL-CM3 4.365925398507543
rcp45 GISS-E2-R 5.093127584906455
rcp45 HadGEM2-ES 3.975100794540447
rcp45 inmcm4 5.611920762843823
rcp45 IPSL-CM5A-LR 4.046193268702261
rcp45 MIROC-ESM 2.597623463346581
rcp45 MIROC5 4.013671767932357
rcp45 MPI-ESM-LR 5.380538460155298
rcp45 MRI-CGCM3 2.9149614988623704
rcp45 NorESM1-M 5.106848211317646
---------------
rcp45 4.345379154759556
---------------
rcp85 bcc-csm1-1 1.94904138550167
rcp85 CanESM2 6.8603425498121435
rcp85 CCSM4 7.318572675846197
rcp85 CNRM-CM5 9.140996341734358
rcp85 CSIRO-Mk3-6-0 4.013237601574417
rcp85 GFDL-CM3 7.5793706827573875
rcp85 GISS-E2-R 7.266524985672345
rcp85 HadGEM2-ES 6.821868384017686
rcp85 inmcm4 6.704833676087224
rcp85 IPSL-CM5A-LR 4.2279087181859785
rcp85 MIROC-ESM 3.4341570493644356
rcp85 MIROC5 6.109857200763656
rcp85 MPI-ESM-LR 7.440037292